# AlexNet
## warm_1: 490장, cool_1: 186장 
## 총 676장

## train data, validation data 6:4 비율

In [3]:
from PIL import Image
import numpy as np
import glob
import os

print("PIL 라이브러리 import 완료!")

PIL 라이브러리 import 완료!


## Step 1. 데이터 불러오고, 웜톤 0, 쿨톤 1로 레이블링

In [4]:
def load_data(img_path, number_of_data=676):  # 웜톤 490 + 쿨톤 186
    # 웜톤 : 0, 쿨톤 : 1
    img_size=224
    color=3
    #이미지 데이터와 라벨(웜톤 : 0, 쿨톤 : 1) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    warm_files = list(glob.iglob(img_path + '/warm_1/*.jpg')) + list(glob.iglob(img_path + '/warm_1/*.JPG'))
    for file in warm_files:
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 웜톤 : 0
        idx=idx+1

    cool_files = list(glob.iglob(img_path + '/cool_1/*.jpg')) + list(glob.iglob(img_path + '/cool_1/*.JPG'))
    for file in cool_files:
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 쿨톤 : 1
        idx=idx+1  
    
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/aiffelthon"
(x_train, y_train)=load_data(image_dir_path)

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 676 입니다.
x_train shape: (676, 224, 224, 3)
y_train shape: (676,)


## Step 2. train data, validation data 나누기, 정규화

In [5]:
import tensorflow as tf
from tensorflow import keras

In [6]:
from sklearn.model_selection import train_test_split

# x_train과 y_train을 훈련 세트와 검증 세트로 분할
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.4, random_state=42)

In [7]:
x_train.dtype

dtype('int32')

In [8]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')

In [9]:
x_train /= 255
x_val /= 255

## Step 3. AlexNet 모델 쌓기

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [15]:
# AlexNet 모델 생성
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# 3rd Fully Connected Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
activation_18 (Activation)   (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_19 (Activation)   (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 6, 6, 384)        

In [17]:
# Compile the model
model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=["accuracy"])

### 시도 1. epochs=10, batch_size=32

In [23]:
hist = model.fit(x=x_train, y=y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val), verbose=1)

Epoch 1/10
13/13 [==============================] - 1s 81ms/step - loss: 0.6039 - accuracy: 0.7086 - val_loss: 0.5666 - val_accuracy: 0.7491
Epoch 2/10
13/13 [==============================] - 1s 40ms/step - loss: 0.6040 - accuracy: 0.7086 - val_loss: 0.5653 - val_accuracy: 0.7491
Epoch 3/10
13/13 [==============================] - 1s 41ms/step - loss: 0.6074 - accuracy: 0.7086 - val_loss: 0.5701 - val_accuracy: 0.7491
Epoch 4/10
13/13 [==============================] - 1s 40ms/step - loss: 0.6040 - accuracy: 0.7086 - val_loss: 0.5695 - val_accuracy: 0.7491
Epoch 5/10
13/13 [==============================] - 1s 42ms/step - loss: 0.6064 - accuracy: 0.7086 - val_loss: 0.5683 - val_accuracy: 0.7491
Epoch 6/10
13/13 [==============================] - 1s 41ms/step - loss: 0.6050 - accuracy: 0.7086 - val_loss: 0.5647 - val_accuracy: 0.7491
Epoch 7/10
13/13 [==============================] - 1s 41ms/step - loss: 0.6051 - accuracy: 0.7086 - val_loss: 0.5710 - val_accuracy: 0.7491
Epoch 8/10
13

In [24]:
test_score=model.evaluate(x_val, y_val)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1]*100))

9/9 [==============================] - 0s 16ms/step - loss: 0.5679 - accuracy: 0.7491
Test loss 0.5679, accuracy 74.91%


### 시도 2. epochs=20, batch_size=32

In [25]:
hist = model.fit(x=x_train, y=y_train, epochs=20, batch_size=32, validation_data=(x_val, y_val), verbose=1)

Epoch 1/20
13/13 [==============================] - 1s 76ms/step - loss: 0.6097 - accuracy: 0.7086 - val_loss: 0.5709 - val_accuracy: 0.7491
Epoch 2/20
13/13 [==============================] - 1s 41ms/step - loss: 0.6095 - accuracy: 0.7086 - val_loss: 0.5717 - val_accuracy: 0.7491
Epoch 3/20
13/13 [==============================] - 1s 41ms/step - loss: 0.6063 - accuracy: 0.7086 - val_loss: 0.5668 - val_accuracy: 0.7491
Epoch 4/20
13/13 [==============================] - 1s 41ms/step - loss: 0.6092 - accuracy: 0.7086 - val_loss: 0.5671 - val_accuracy: 0.7491
Epoch 5/20
13/13 [==============================] - 1s 41ms/step - loss: 0.6074 - accuracy: 0.7086 - val_loss: 0.5686 - val_accuracy: 0.7491
Epoch 6/20
13/13 [==============================] - 1s 44ms/step - loss: 0.6093 - accuracy: 0.7086 - val_loss: 0.5722 - val_accuracy: 0.7491
Epoch 7/20
13/13 [==============================] - 1s 42ms/step - loss: 0.6071 - accuracy: 0.7086 - val_loss: 0.5668 - val_accuracy: 0.7491
Epoch 8/20
13

In [26]:
test_score=model.evaluate(x_val, y_val)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1]*100))

9/9 [==============================] - 0s 16ms/step - loss: 0.5689 - accuracy: 0.7491
Test loss 0.5689, accuracy 74.91%


### 시도 3. early_stopping 적용, epochs=50, batch_size=32

In [28]:
from keras.callbacks import EarlyStopping

# EarlyStopping 객체를 생성합니다.
# 'val_loss'를 모니터링하고, 3 epoch 동안 성능 향상이 없으면 학습을 중단합니다.
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# fit 함수를 호출할 때 callbacks 매개변수에 early_stopping을 전달합니다.
hist = model.fit(x=x_train, y=y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val), verbose=1, callbacks=[early_stopping])


Epoch 1/50
13/13 [==============================] - 1s 81ms/step - loss: 0.6031 - accuracy: 0.7086 - val_loss: 0.5682 - val_accuracy: 0.7491
Epoch 2/50
13/13 [==============================] - 1s 42ms/step - loss: 0.6091 - accuracy: 0.7086 - val_loss: 0.5679 - val_accuracy: 0.7491
Epoch 3/50
13/13 [==============================] - 1s 42ms/step - loss: 0.6013 - accuracy: 0.7086 - val_loss: 0.5651 - val_accuracy: 0.7491
Epoch 4/50
13/13 [==============================] - 1s 42ms/step - loss: 0.6074 - accuracy: 0.7086 - val_loss: 0.5674 - val_accuracy: 0.7491
Epoch 5/50
13/13 [==============================] - 1s 41ms/step - loss: 0.6039 - accuracy: 0.7086 - val_loss: 0.5686 - val_accuracy: 0.7491
Epoch 6/50
13/13 [==============================] - 1s 42ms/step - loss: 0.6039 - accuracy: 0.7086 - val_loss: 0.5700 - val_accuracy: 0.7491


In [29]:
test_score=model.evaluate(x_val, y_val)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1]*100))

9/9 [==============================] - 0s 16ms/step - loss: 0.5700 - accuracy: 0.7491
Test loss 0.5700, accuracy 74.91%


### 시도 4. early_stopping 적용, epochs=50, batch_size=64

In [30]:
from keras.callbacks import EarlyStopping

# EarlyStopping 객체를 생성합니다.
# 'val_loss'를 모니터링하고, 3 epoch 동안 성능 향상이 없으면 학습을 중단합니다.
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# fit 함수를 호출할 때 callbacks 매개변수에 early_stopping을 전달합니다.
hist = model.fit(x=x_train, y=y_train, epochs=50, batch_size=64, validation_data=(x_val, y_val), verbose=1, callbacks=[early_stopping])


Epoch 1/50
7/7 [==============================] - 1s 134ms/step - loss: 0.6047 - accuracy: 0.7086 - val_loss: 0.5678 - val_accuracy: 0.7491
Epoch 2/50
7/7 [==============================] - 0s 70ms/step - loss: 0.6038 - accuracy: 0.7086 - val_loss: 0.5659 - val_accuracy: 0.7491
Epoch 3/50
7/7 [==============================] - 0s 69ms/step - loss: 0.6019 - accuracy: 0.7086 - val_loss: 0.5651 - val_accuracy: 0.7491
Epoch 4/50
7/7 [==============================] - 0s 69ms/step - loss: 0.6035 - accuracy: 0.7086 - val_loss: 0.5670 - val_accuracy: 0.7491
Epoch 5/50
7/7 [==============================] - 0s 68ms/step - loss: 0.6041 - accuracy: 0.7086 - val_loss: 0.5683 - val_accuracy: 0.7491
Epoch 6/50
7/7 [==============================] - 0s 67ms/step - loss: 0.6058 - accuracy: 0.7086 - val_loss: 0.5678 - val_accuracy: 0.7491


In [31]:
test_score=model.evaluate(x_val, y_val)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1]*100))

9/9 [==============================] - 0s 16ms/step - loss: 0.5678 - accuracy: 0.7491
Test loss 0.5678, accuracy 74.91%


## Step 4. 한 줄 정리

시도 1. epochs=10, batch_size=32
loss: 0.5679 - accuracy: 0.7491

시도 2. epochs=20, batch_size=32
loss: 0.5689 - accuracy: 0.7491

시도 3. early_stopping 적용, epochs=50, batch_size=32
loss: 0.5700 - accuracy: 0.7491

**시도 4. early_stopping 적용, epochs=50, batch_size=64
loss: 0.5678 - accuracy: 0.7491**